# Memoris project

## Imports

#### Internal modules

In [ ]:
from utils.io import dataframe_viewer
from utils.orm import orm_boreholes_from_dataframe
from utils.visual import legend_from_attributes, find_new_word_for_legend
from utils.config import DEFAULT_LITHO_LEXICON, DEFAULT_LITHO_LEGEND, DEFAULT_BOREHOLE_LEGEND, DEFAULT_POL_LEXICON

from core.project import Project
from definitions import ROOT_DIR

#### External modules

In [ ]:
import os
import re
import pandas as pd
import pyvista as pv

## Reading of CSV files in a dir

In [ ]:
data_dir = ROOT_DIR + '/CF_data/Result_traitem/fusion_finale/filtered/' 

In [ ]:
litho_file = data_dir + 'Lithologies.csv'
sample_file = data_dir + 'Samples.csv'
bh_file = data_dir + 'Boreholes.csv'

In [ ]:
litho = pd.read_csv(litho_file, sep=',')
sample = pd.read_csv(sample_file, sep=',')

In [ ]:
dataframe_viewer(litho, rows=5, un_val='ID'), dataframe_viewer(sample, rows=5, un_val='ID')

In [ ]:
litho.loc[1573, 'Litho_base'] = 4.5
litho['Type_ech'] = 'Sol'

### Keep data concerning Memoris site only

In [ ]:
data_list = [litho, sample]
x_lim = (152804.75, 152921.12)
y_lim = (122545.08, 122667.38)
qry = f'X>={x_lim[0]} and X<={x_lim[1]} and Y>={y_lim[0]} and Y<={y_lim[1]}'

final_df = pd.DataFrame()
last_index = None
skip_cols = []
for dataf in data_list:
    assert isinstance(dataf, pd.DataFrame)
    df = dataf.query(qry).copy()

    # rename certain columns' name
    for col in df.columns:
        if col not in skip_cols:
            if re.search('top|toit', col, re.I):
                df.rename(columns={col: 'Top_intv'}, inplace=True)
            elif re.search('base|mur|assise', col, re.I):
                df.rename(columns={col: 'Base_intv'}, inplace=True)
            elif re.search('thick|epais', col, re.I):
                df.rename(columns={col: 'Thick_intv'}, inplace=True)
            elif re.search('descr', col, re.I):
                df.rename(columns={col: 'Descr_intv'}, inplace=True)

    if last_index is not None:
        df.index = range(last_index, last_index + len(df))

    final_df = final_df.append(df, ignore_index=True)
    last_index = len(final_df)
print(last_index, 'rows')

In [ ]:
memoris_site_data = final_df

### Rename boreholes by synonyms

In [ ]:
from utils.io import dict_viewer, find_bh_synonyms, choose_bh_synonym, value_replication

In [ ]:
syn_dict = find_bh_synonyms(memoris_site_data, regex_skip='bloqu|bis$|\d+[a-e]\'?M?$', dist_max=0.2, display=True)

In [ ]:
#choices = {63:'Mon3', 71:'_100_', 72:'_102_', 73:'_103_', 74:'_105_', 75:'_108_'}
memoris_site_data, syn_dict = choose_bh_synonym(memoris_site_data, syn_dict, regex_choice='^F|^P|^_F|^\d+$')#, choice_dict=choices)

In [ ]:
crit_cols = ['ID','X','Y']
#pass_cols = crit_cols + ['Litho_top','Litho_base','Litho_epais','Description']
copy_cols = ['Type', 'Long_for', 'Diam_for'] #list(data.columns)
memoris_site_data = value_replication(memoris_site_data, crit_cols=crit_cols, pass_cols=None, copy_cols=copy_cols)

In [ ]:
cols=['ID', 'Date_for', 'Diam_for', 'Long_for', 'top', 'base', 'Type', 'Descr_intv', 'ID_ech', 'Date_ech']
view_cols = []
for col in memoris_site_data.columns:
    for c in cols:
        if re.search(c, col, re.I):
            view_cols.append(col)

In [ ]:
memoris_site_data = memoris_site_data.sort_values(['ID','Top_intv', 'Base_intv']).reset_index(drop=True)

In [ ]:
bh_id='205'
dataframe_viewer(memoris_site_data.query(f"ID=='{bh_id}'")[view_cols], rows=12, cols=16)
#dataframe_viewer(memoris_site_data.query(f"ID=='{bh_id}'"), rows=10, cols=16)

## Create pollutants list from the dataframe columns name

In [ ]:
pollutants = []
skipped = []
for i, c in enumerate(sample.columns):
    if c in DEFAULT_POL_LEXICON.abbreviations.keys() or c in DEFAULT_POL_LEXICON.abbreviations.values():
        #print(f"{i}: {c}")
        pollutants.append(c)
    else:
        skipped.append(c)
print(pollutants)

## Create Boreholes and components from dataframes

In [ ]:
boreholes_orm, components_orm, link_intv_comp = orm_boreholes_from_dataframe(
                                                data_list=[memoris_site_data], default_z=102, id_col='ID', 
                                                bh_type_col='Type', date_col='Date_for', 
                                                length_col = 'Long_for', diameter_col='Diam_for',
                                                sample_type_col='Type_ech', sample_id_col='ID_ech',
                                                attributes=['lithology']+pollutants, 
                                                lexicons={'lithology':{'lexicon':DEFAULT_LITHO_LEXICON}}
                                                )

In [ ]:
len(boreholes_orm), len(components_orm)

In [ ]:
boreholes_orm # list of core.BoreholeOrm objects

In [ ]:
components_orm # dict of striplog.Component objects

In [ ]:
link_intv_comp

## Create a dictionary containing legends for each representation attribute

In [ ]:
legend_dict = legend_from_attributes([('borehole_type', DEFAULT_BOREHOLE_LEGEND), 
                                      ('lithology', DEFAULT_LITHO_LEGEND)] + pollutants)

## create a project

In [ ]:
p = Project.load(db_name='../tmp_files/Memoris_project', legend_dict=legend_dict, lexicon=DEFAULT_LITHO_LEXICON, 
                 auto_erase_db=True)

### Insert boreholes and components into the project

In [ ]:
p.add_components(components_orm)

In [ ]:
for bhorm in boreholes_orm:
    p.add_borehole(bhorm)

In [ ]:
p.add_link_components_intervals(link_intv_comp)

### Validate modifications 

In [ ]:
p.commit()

In [ ]:
p.refresh(update_3d=True, update_legend=True)

In [ ]:
p.session.close()

## Visualization

### Localization of project boreholes

In [ ]:
p.plot_map()

## Plot a log for one specific borehole, according to the representation attribute

In [ ]:
attributes = list(legend_dict.keys())
print(attributes)

In [ ]:
n = 'lithology' #'borehole_type'
if isinstance(n, str):
    n = attributes.index(n)

print('Chosen attribute is :', attributes[n])
p.repr_attribute = attributes[n]

In [ ]:
p.attrib_legend

In [ ]:
p.attrib_cmap

In [ ]:
print(p.attrib_values)

In [ ]:
find_new_word_for_legend(p.legend_dict, p.repr_attribute)

### Borehole log

In [ ]:
bh_name = '205'

In [ ]:
ticks=(0.1, 0.5)
p.plot_log(bh_name=bh_name, ticks=ticks)

In [ ]:
p.boreholes_3d[bh_name]#.attrib_components()

#### add ground surface to the plotter

In [ ]:
other_vtks = {'ground_surface':{'mesh':pv.read('../20211122/ground_surface.vtk'),
                                'color':'green', 
                                'opacity':0.2}}

In [ ]:
bg_color = 'white'
p.plot_3d(bh_name_size=10, legend_title_size=25, legend_label_size=6, legend_vertical=True, 
          bg_color=bg_color, window_size=(800, 600), diam=0.2, smooth_shading=True,
         add_vtks_obj=other_vtks)

* Change repr_attribute

In [ ]:
n = 'lithology'
if isinstance(n, str):
    n = attributes.index(n)

print('Chosen attribute is :', attributes[n])
p.repr_attribute = attributes[n]

In [ ]:
from striplog import Interval, Component, Striplog

In [ ]:
intv = [Interval(0,1,description=None, components=[Component({'lithology':'sable'})]), 
        Interval(1,2,components=[]), 
        Interval(2,3,components=[Component({'lithology':'limon'})]),
        Interval(3,4,components=[Component({'lithology':'remblai'})])]

In [ ]:
Striplog(intv).plot(aspect=2, ticks=(.5, 1))

In [ ]:
n = 'arsenic'
if isinstance(n, str):
    n = attributes.index(n)

print('Chosen attribute is :', attributes[n])
p.repr_attribute = attributes[n]

In [ ]:
ticks=(0.05, 0.2)
p.plot_log(bh_name=bh_name, ticks=ticks)

In [ ]:
p.boreholes_3d[bh_name]#.attrib_components()

In [ ]:
p.attrib_legend

In [ ]:
p.attrib_cmap

In [ ]:
p.repr_attribute

In [ ]:
p.boreholes_3d[bh_name].attrib_components()

In [ ]:
p.__fictive_bh3d__.plot_log()

### 3D display of project's boreholes

### 3D display of one borehole